# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model
- Compare and contrast model validation strategies

## Let's Get Started

We included the code to pre-process the Ames Housing dataset below. This is done for the sake of expediency, although it may result in data leakage and therefore overly optimistic model metrics.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

## Train-Test Split

Perform a train-test split with a test set of 20% and a random state of 4.

In [2]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split

In [4]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Fit a Model

Fit a linear regression model on the training set

In [8]:
# Import LinearRegression from sklearn.linear_model
from sklearn.linear_model import LinearRegression

In [9]:
# Instantiate and fit a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

### Calculate MSE

Calculate the mean squared error on the test set

In [12]:
# Import mean_squared_error from sklearn.metrics
from sklearn.metrics import mean_squared_error

In [15]:
# Calculate MSE on test set
y_hat = model.predict(X_test)

mean_squared_error(y_test, y_hat)

0.17595331097301373

## Cross-Validation using Scikit-Learn

Now let's compare that single test MSE to a cross-validated test MSE.

In [18]:
# Import cross_val_score from sklearn.model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

In [24]:
# Find MSE scores for a 5-fold cross-validation
MSEs = cross_val_score(model, X, y, cv=5, scoring=make_scorer(mean_squared_error))

In [25]:
import numpy as np

np.mean(MSEs)

0.1770283421000112

Compare and contrast the results. What is the difference between the train-test split and cross-validation results? Do you "trust" one more than the other?

In [26]:
# Your answer here

# In this particular case, both methods have slightly different results, with K-Fold measuring a marginally higher Mean Squared Error.
# I trust the K-Fold process a bit more, since it is resistant to "luck" when splitting the dataset. Train-test split is very vulnerable
# to obtaining different results depending on the split.

## Level Up: Let's Build It from Scratch!

### Create a Cross-Validation Function

Write a function `kfolds(data, k)` that splits a dataset into `k` evenly sized pieces. If the full dataset is not divisible by `k`, make the first few folds one larger then later ones.

For example, if you had this dataset:

In [27]:
example_data = pd.DataFrame({
    "color": ["red", "orange", "yellow", "green", "blue", "indigo", "violet"]
})
example_data

,color
0,red
1,orange
2,yellow
3,green
4,blue
5,indigo
6,violet


`kfolds(example_data, 3)` should return:

* a dataframe with `red`, `orange`, `yellow`
* a dataframe with `green`, `blue`
* a dataframe with `indigo`, `violet`

Because the example dataframe has 7 records, which is not evenly divisible by 3, so the "leftover" 1 record extends the length of the first dataframe.

In [28]:
def kfolds(data, k):
    folds = []
    
    # Your code here
    base_elements = len(data) // k
    extra_elements = len(data) % k

    for i in range(k):

        if(i < extra_elements):
            split = data[:base_elements + 1]
            data = data[base_elements + 1:]
        else:
            split = data[:base_elements]
            data = data[base_elements:]

        folds.append(pd.DataFrame(split))
    
    return folds

In [29]:
results = kfolds(example_data, 3)
for result in results:
    print(result, "\n")

    color
0     red
1  orange
2  yellow 

   color
3  green
4   blue 

    color
5  indigo
6  violet 



### Apply Your Function to the Ames Housing Data

Get folds for both `X` and `y`.

In [30]:
# Apply kfolds() to ames_data with 5 folds
xfolds = kfolds(X, 5)
yfolds = kfolds(y, 5)

### Perform a Linear Regression for Each Fold and Calculate the Test Error

Remember that for each fold you will need to concatenate all but one of the folds to represent the training data, while the one remaining fold represents the test data.

In [33]:
# Replace None with appropriate code
test_errs = []
k = 5

for n in range(k):
    # Split into train and test for the fold
    X_train = pd.concat([fold for i, fold in enumerate(xfolds) if i != n])
    X_test = xfolds[n]
    y_train = pd.concat([fold for i, fold in enumerate(yfolds) if i != n])
    y_test = yfolds[n]
    
    # Fit a linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Evaluate test errors
    y_hat = model.predict(X_test)
    mse = mean_squared_error(y_test, y_hat)

    test_errs.append(mse)

print(test_errs)

[0.12431546148437424, 0.19350064631313132, 0.18910530431311184, 0.17079325250026917, 0.20742704588916946]


If your code was written correctly, these should be the same errors as scikit-learn produced with `cross_val_score` (within rounding error). Test this out below:

In [34]:
# Compare your results with sklearn results
cross_val_score(model, X, y, cv=5, scoring=make_scorer(mean_squared_error))

array([0.12431546, 0.19350065, 0.1891053 , 0.17079325, 0.20742705])

This was a bit of work! Hopefully you have a clearer understanding of the underlying logic for cross-validation if you attempted this exercise.

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are more robust than train-test split.